#1. Ejercicio de clasificación con algoritmo Naive Bayes
#Urban Land Cover Data Set

Number of instances 168

Number of Attributes 148

Contiene datos de entrenamiento y prueba para clasificar una imagen aérea de alta resolución en 9 tipos de cobertura de suelo urbano. Para la clasificación se utiliza información espectral, de tamaño, forma y textura de múltiples escalas. 
 El conjunto de datos de prueba proviene de una muestra aleatoria de la imagen.

Las clases de cobertura terrestre son:


* Árboles
* Césped
* Suelo
* Concreto
* Asfalto
* Edificios
* Coches
* Piscinas
* Sombras





#2. Importación de librerías y montando google drive

In [ ]:
import ____ as np
import matplotlib.pyplot as ____
import ____ as pd
import seaborn as ____
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn._____ import LabelEncoder
from sklearn.preprocessing import ____, ____
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import KFold
from sklearn._____ import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from collections import Counter
import itertools

# 3. Lectura y visualuzación del dataset




In [ ]:
path = ______
train_path = os.path.join(path,'training.csv')
test_path = os.path.join(path,'testing.csv')


In [ ]:
train_df = ______(train_path)
test_df = pd.read_csv(______)
clases = train_df['label'].unique()
train_df.dropna(axis=0, inplace=True)
train_df

##Balance de clases 

In [ ]:
print('Número de instancias por clase, conjunto de entrenamiento')
print(______['label'].value_counts())

In [ ]:
f,ax=plt.subplots(1,2,figsize=(16,5))

sns.countplot('label',data=train_df,ax=ax[0])
ax[0].set_title('Distribution of Classes TRAIN')

sns.countplot('label',data=test_df,ax=ax[1])
ax[1].set_title('Distribution of Classes TEST')

ax[0].set_ylim(0,100)
ax[1].set_ylim(0,100)

plt.show()

In [ ]:
print ('Conteo de valores faltantes: \n')
print (______.isna().sum().sum())

#4. Escalamiento

In [ ]:
scaler =______()
train_df.loc[:, train_df.columns != 'label'] = scaler.fit_transform(train_df.loc[:, train_df.columns != 'label'])
test_df.loc[:, test_df.columns != 'label'] = scaler.transform(test_df.loc[:, test_df.columns != 'label'])


#5. Codificación de etiquetas

In [ ]:
le = ______()
train_df['label'] = le.______(train_df.label.values)
test_df['label'] = le.______(test_df.label.values)

In [ ]:
train_df

## 6. Selección de características por análisis de correlación (Coeficiente de Pearson)

In [ ]:
#"todas con todas"
correlations = train_df.corr().abs().unstack().sort_values(kind="quicksort").reset_index() 
correlations

In [ ]:
#remueve las que son consigo mismas
correlations = correlations[correlations['level_0'] != correlations['level_1']] 
correlations

In [ ]:
#identificar las que tienen correlación mayor a 0.99
correlations = correlations.loc[correlations[0] >= 0.99]  
#correlations

In [ ]:
removable_features = set(list(correlations['level_1']))
len(removable_features)
#removable_features

Eliminar las removable features a los conjuntos de train y test

In [ ]:
train_df = train_df.drop(______, axis=______)
train_df

In [ ]:
test_df = test_df.drop(removable_features, axis=1)
test_df

#7. Modelo de clasificación con Naive Bayes y k-fold CV




In [ ]:
X_train = train_df.values[:,1:]
Y_train = train_df.values______

X_test = test_df.values______
Y_test = test_df.values[:,0]

In [ ]:
print(f'Dimension de los datos de entrenamiento {X_train.shape}')
print(f'Dimension de los datos de prueba {X_test.shape}')

In [ ]:
seed = 42
from sklearn.naive_bayes import GaussianNB 
NB =  GaussianNB()
kfold = KFold(n_splits=______, random_state=seed, shuffle=True)

In [ ]:
score_train = cross_val_score(NB, ______, ______, cv=kfold, scoring="accuracy")
print("Train Acc: %s"%score_train)

In [ ]:
y_pred = cross_val_predict(NB, ______, ______, cv=10)

score = metrics.accuracy_score(Y_test, y_pred)
print("Test Acc: %s"%score)

In [ ]:
#Funciones para MC y métricas
def plot_confusion_matrix(cm, classes, tit, normalize=False):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)
        title, fmt = 'Matriz de confusión normalizada', '.2f'
    else:
        title, fmt = tit, 'd'
    plt.figure(figsize=(10,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)#, fontsize=12)
    plt.colorbar(pad=0.05)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=40)
    plt.yticks(tick_marks, classes)
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center", 
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Clase Verdadera')#, fontsize=10)
    plt.xlabel('Clase Predicha')#, fontsize=10)
    plt.savefig(title+'.png')
    #plt.grid(b=None)
    plt.show()
def sens_spec(cls_names, y_true, y_pred, pesos):
  sensitivity = []
  specificity = []
  acc=[]
  for i,name in enumerate(cls_names):
    TP = np.sum((y_true==name) & (y_pred==name))
    TN = np.sum((y_true!=name) & (y_pred!=name))
    FP = np.sum((y_true!=name) & (y_pred==name))
    FN = np.sum((y_true==name) & (y_pred!=name))
    sensitivity.append(TP/(TP+FN))
    specificity.append(FP/(TN+FP))
    acc.append(TP/(TP+FP))
  sensitivity.append(sum([x*y for x,y in zip(sensitivity,pesos)]))
  specificity.append(sum([x*y for x,y in zip(specificity,pesos)]))
  acc.append(sum([x*y for x,y in zip(acc,pesos)]))
  return sensitivity, specificity, acc

Matriz de confusión

In [ ]:
n_clases = 9
predictions = np.float32(y_pred)
true_labels = np.float32(Y_test)
cnf_matrix = confusion_matrix(true_labels, predictions, labels=range(n_clases))
tit = 'Matriz de confusión NB'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=False)

Pesos por clase

In [ ]:
pesos = train_df['label'].value_counts().sort_index().tolist()/ np.sum(train_df['label'].value_counts().tolist())
print(clases)
print(pesos)

In [ ]:
sensitivity, specificity, acc = sens_spec(range(9), true_labels, predictions, pesos)
d = {'Sensitivity':sensitivity, 'Specificity':specificity, 'Accuracy':acc}
ind = clases.tolist()+['Promedio']
df = pd.DataFrame(d, index=ind)
index = df.index
precprom = sum([x*y for x,y in zip(acc,pesos)])
index.name = 'Acc Pond: %s'%precprom
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df, annot=True)